In [1]:
import sys

In [2]:
sys.path.append('/opt/opencv-3.0.0/release/lib/python2.7/site-packages/')

In [3]:
sys.path.append('/home/fmpaezri/repos/localization-agent/')

In [4]:
import pickle
import numpy
from PIL import Image
import IPython.display
import os

import tracking.Tester
import tracking.RecurrentTracker
import tracking.TheanoGruRnn
import tracking.VideoSequence as vs
import tracking.VisualAttention
import tracking.GaussianGenerator

Using gpu device 0: Graphics Device (CNMeM is disabled, CuDNN 3007)


In [5]:
cd ../tracking

/home/fmpaezri/repos/localization-agent/tracking


In [6]:
with open('/data1/vot-challenge/experiments/ECCV_evaluation/expD01/model.pkl', 'r') as modelFile:
    %time rnn = pickle.load(modelFile)

CPU times: user 15.8 s, sys: 468 ms, total: 16.3 s
Wall time: 16.4 s


In [7]:
tracker = tracking.RecurrentTracker.RecurrentTracker(None, rnn)

In [8]:
tester = tracking.Tester.Tester(tracker)

In [9]:
#samplesDir = '/tmp/sampleSequences'

#gg = tracking.GaussianGenerator.GaussianGenerator(
#    '/home/fmpaezri/datasets/mscoco/',
#    '/home/fmpaezri/repos/localization-agent/summaries/cocoTrainSummaryCategAndSideGt100SmplsAllCorrected.pkl',
#    ['random', 'sine', 'stretched'], ['stretched'], None, grayscale=False, 
#    objectPathTemplate='./train2014', scenePathTemplate='./train2014'
#        )

#%time gg.saveBatch(4, 0, 60, )

In [10]:
sequencePath = '/home/datasets/datasets1/vot-challenge/sequences2014/car/'
#sequencePath = os.path.join(samplesDir, numpy.random.choice(os.listdir(samplesDir)))

In [11]:
%time data, labels = tester.loadImageSequence(sequencePath, newShape=(rnn.imgSize,rnn.imgSize))

CPU times: user 876 ms, sys: 8 ms, total: 884 ms
Wall time: 883 ms


In [12]:
#ONLY FOR VOT BOXES
#Correcting box ccords as expected by IoU
labelsCopy = labels.copy()
labelsCopy[:,0] = labels[:,0::2].min(axis=1) 
labelsCopy[:,1] = labels[:,1::2].min(axis=1)
labelsCopy[:,2] = labels[:,0::2].max(axis=1) 
labelsCopy[:,3] = labels[:,1::2].max(axis=1)
labels = labelsCopy

In [13]:
data = data[numpy.newaxis, ...].astype(numpy.float32)

In [14]:
labels = labels[numpy.newaxis,...]

In [15]:
print data.shape, labels.shape

(1, 252, 224, 224, 3) (1, 252, 4)


In [16]:
%time iou, predLabels = tester.test(data, labels, None, rnn.batchSize, rnn.imgSize, False, labels.shape[1], 4, '/tmp')

CPU times: user 17.3 s, sys: 3.99 s, total: 21.3 s
Wall time: 21.4 s


In [17]:
iou.mean()

0.063473161019772933

In [18]:
seq = vs.VideoSequence((Image.fromarray(frame.astype(numpy.uint8)) for frame in data[0]))

In [19]:
seq.addBoxes(labels[0], 'green')
seq.addBoxes((predLabels[0]), 'red')

In [20]:
outputName = '/tmp/testSequence1.mp4'
seq.exportToVideo(30, outputName)

In [21]:
IPython.display.HTML(vs.displayHTML(outputName))